In [10]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade accelerate

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import and Setup

import pandas as pd
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import numpy as np
import fightinwords as fw
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import requests
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import StratifiedKFold
import random
from sklearn.model_selection import cross_val_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns


## Right and Left Labels Creation From Congress Bills & Presidential Debates

### Scraping Presidential Debates

In [13]:
#Scraped Data for Vice Presidential Election 2020 (Kamala Harris and Mike Pence)

URL = "https://debates.org/voter-education/debate-transcripts/vice-presidential-debate-at-the-university-of-utah-in-salt-lake-city-utah/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the indices of paragraphs containing the word "abortion"
abortion_indices = [index for index, p in enumerate(all_paragraphs) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Extract paragraphs between the first and last mention of "abortion"
    start_index = abortion_indices[0]
    end_index = abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'dem/rep', and 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}
    count_pence = 0

    for paragraph in paragraphs_between_abortion:
        count_pence += paragraph.get_text().count('PENCE:')
        
        if count_pence == 4:
            # Modify the paragraph in-place if it's the fourth instance of 'PENCE:'
            paragraph.string = re.sub(r'^PENCE:', 'HARRIS:', paragraph.get_text())
    
        # Skip paragraphs that start with "PAGE:"
        if not paragraph.get_text().startswith("PAGE:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'PENCE:' in paragraph.get_text():
                data['name'].append('PENCE')
                data['year'].append(2020)
                data['dem/rep'].append('rep')
                data['left/right'].append('right')
            else:
                data['name'].append('HARRIS')
                data['year'].append(2020)
                data['dem/rep'].append('dem')
                data['left/right'].append('left')
            
            data['content'].append(re.sub(r'^PENCE:|HARRIS:', '', paragraph.get_text()))

    prez_df_2020 = pd.DataFrame(data)


#Scraped Data for Vice Presidential Election 2016 (Mike Pence and Tim Kaine)

#October 4, 2016

URL = "https://www.debates.org/voter-education/debate-transcripts/october-4-2016-debate-transcript/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the index of the paragraph with the last mention of "abortion"
last_abortion_index = max([index for index, p in enumerate(all_paragraphs) if 'abortion' in p.get_text().lower()], default=0)

# Find the index of the paragraph starting with "PENCE: But for me"
pence_index = next((i for i, p in enumerate(all_paragraphs) if p.get_text().startswith("PENCE: But for me")), None)

# Extract paragraphs between "PENCE: But for me" and the last mention of "abortion"
paragraphs_selected = all_paragraphs[pence_index:last_abortion_index + 1]

# Create a DataFrame with columns for 'name', 'dem/rep', 'left/right', and 'content'
data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

current_speaker = None

for paragraph in paragraphs_selected:
    text = paragraph.get_text()
    
    # Skip paragraphs that start with "QUIJANO:"
    if text.startswith("QUIJANO:"):
        continue
    
    # Determine 'name' and 'dem/rep' based on the speaker
    if 'PENCE:' in text:
        current_speaker = 'PENCE'
        dem_rep = 'rep'
    elif 'KAINE:' in text:
        current_speaker = 'KAINE'
        dem_rep = 'dem'
    
    # Determine 'left/right' based on the speaker
    if 'PENCE:' in text:
        left_right = 'right'
    elif 'KAINE:' in text:
        left_right = 'left'
    
    # Append data to the dictionary
    data['name'].append(current_speaker)
    data['year'].append(2016)
    data['dem/rep'].append(dem_rep)
    data['left/right'].append(left_right)
    data['content'].append(re.sub(r'^PENCE:|KAINE:', '', paragraph.get_text()))
    
# Create a DataFrame from the dictionary
prez_df_2016 = pd.DataFrame(data)






#Scraped Data for Third residential Election 2016 (Hilary Clinton and Donald Trump) 

#October 19, 2016

URL = "https://www.debates.org/voter-education/debate-transcripts/october-19-2016-debate-transcript/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the indices of paragraphs containing the word "abortion"
abortion_indices = [index for index, p in enumerate(all_paragraphs) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Extract paragraphs between the first and last mention of "abortion"
    start_index = abortion_indices[0]
    end_index = abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "WALLACE:"
        if not paragraph.get_text().startswith("WALLACE:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'TRUMP:' in paragraph.get_text():
                current_speaker = 'TRUMP'
            elif 'CLINTON:' in paragraph.get_text():
                current_speaker = 'CLINTON'
            
            data['name'].append(current_speaker)
            data['year'].append(2016)
            data['dem/rep'].append('rep' if current_speaker == 'TRUMP' else 'dem')
            data['left/right'].append('right' if current_speaker == 'TRUMP' else 'left')
            data['content'].append(re.sub(r'^TRUMP:|CLINTON:', '', paragraph.get_text()))

    prez_df_2016_2 = pd.DataFrame(data)




#Scraped Data for Vice Presidential Election 2012 (Biden and Ryan)

#October 11, 2012

URL = "https://www.debates.org/voter-education/debate-transcripts/october-11-2012-the-biden-romney-vice-presidential-debate/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the index of the paragraph containing "RYAN: I don’t see how a"
ryan_paragraph_index = next((index for index, p in enumerate(all_paragraphs) if 'RYAN: I don’t see how a' in p.get_text()), None)

# Find the indices of paragraphs containing the word "abortion" after the specified paragraph
abortion_indices = [index for index, p in enumerate(all_paragraphs[ryan_paragraph_index:]) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Adjust the start index to include the paragraph containing "RYAN: I don’t see how a"
    start_index = ryan_paragraph_index
    end_index = ryan_paragraph_index + abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "RADDATZ:"
        if not paragraph.get_text().startswith("RADDATZ:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'RYAN:' in paragraph.get_text():
                current_speaker = 'RYAN'
            elif 'BIDEN:' in paragraph.get_text():
                current_speaker = 'BIDEN'
            
            data['name'].append(current_speaker)
            data['year'].append(2012)
            data['dem/rep'].append('rep' if current_speaker == 'RYAN' else 'dem')
            data['left/right'].append('right' if current_speaker == 'RYAN' else 'left')
            data['content'].append(re.sub(r'^RYAN:|BIDEN:', '', paragraph.get_text()))

    prez_df_2012 = pd.DataFrame(data)



#Scraped Data for Presidential Election 2008 (Third McCain and Obama Debate)

#October 15, 2008

URL = "https://www.debates.org/voter-education/debate-transcripts/october-15-2008-debate-transcript/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the indices of paragraphs containing the word "abortion"
abortion_indices = [index for index, p in enumerate(all_paragraphs) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Extract paragraphs between the first and last mention of "abortion"
    start_index = abortion_indices[0]
    end_index = abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "SCHIEFFER::"
        if not paragraph.get_text().startswith("SCHIEFFER:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'MCCAIN:' in paragraph.get_text():
                current_speaker = 'MCCAIN'
            elif 'OBAMA:' in paragraph.get_text():
                current_speaker = 'OBAMA'
            
            data['name'].append(current_speaker)
            data['year'].append(2008)
            data['dem/rep'].append('rep' if current_speaker == 'MCCAIN' else 'dem')
            data['left/right'].append('right' if current_speaker == 'MCCAIN' else 'left')
            data['content'].append(re.sub(r'^MCCAIN:|OBAMA:', '', paragraph.get_text()))

    prez_df_2008 = pd.DataFrame(data)


#Scraped Data for Presidential Election 2004 (Bush and Kerry)

#October 13, 2004

URL = "https://www.debates.org/voter-education/debate-transcripts/october-13-2004-debate-transcript/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the index of the paragraph containing "RYAN: I don’t see how a"
kerry_paragraph_index = next((index for index, p in enumerate(all_paragraphs) if 'KERRY: I respect their views.' in p.get_text()), None)

# Find the indices of paragraphs containing the word "abortion" after the specified paragraph
abortion_indices = [index for index, p in enumerate(all_paragraphs[kerry_paragraph_index:]) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Adjust the start index to include the paragraph containing "KERRY: I respect their views."
    start_index = kerry_paragraph_index
    end_index = kerry_paragraph_index + abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "SCHIEFFER"
        if not paragraph.get_text().startswith("SCHIEFFER"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'BUSH:' in paragraph.get_text():
                current_speaker = 'BUSH'
            elif 'KERRY:' in paragraph.get_text():
                current_speaker = 'KERRY'
            
            data['name'].append(current_speaker)
            data['year'].append(2004)
            data['dem/rep'].append('rep' if current_speaker == 'BUSH' else 'dem')
            data['left/right'].append('right' if current_speaker == 'BUSH' else 'left')
            data['content'].append(re.sub(r'^BUSH:|KERRY:', '', paragraph.get_text()))

    prez_df_2004 = pd.DataFrame(data)




#Scraped Data for Presidential Election 2004 (Bush and Kerry)

#October 8, 2004

URL = "https://www.debates.org/voter-education/debate-transcripts/october-8-2004-debate-transcript/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the index of the paragraph containing "RYAN: I don’t see how a"
starting_index = next((index for index, p in enumerate(all_paragraphs) if 'DEGENHART: Senator Kerry, suppose' in p.get_text()), None)

# Find the indices of paragraphs containing the word "abortion" after the specified paragraph
abortion_indices = [index for index, p in enumerate(all_paragraphs[starting_index:]) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    end_index = starting_index + abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[starting_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "DEGENHART:"
        if not paragraph.get_text().startswith("DEGENHART:") and not paragraph.get_text().startswith("GIBSON:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'BUSH:' in paragraph.get_text():
                current_speaker = 'BUSH'
            elif 'KERRY:' in paragraph.get_text():
                current_speaker = 'KERRY'
            
            data['name'].append(current_speaker)
            data['year'].append(2004)
            data['dem/rep'].append('rep' if current_speaker == 'BUSH' else 'dem')
            data['left/right'].append('right' if current_speaker == 'BUSH' else 'left')
            data['content'].append(re.sub(r'^BUSH:|KERRY:', '', paragraph.get_text()))

    prez_df_2004_2 = pd.DataFrame(data)



#Scraped Data for Presidential Election 2000 (Gore and Bush)

#October 8, 2004

URL = "https://www.debates.org/voter-education/debate-transcripts/october-3-2000-transcript/" 
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_paragraphs = soup.find_all('p')

# Find the indices of paragraphs containing the word "abortion"
abortion_indices = [index for index, p in enumerate(all_paragraphs) if 'abortion' in p.get_text().lower()]

if abortion_indices:
    # Extract paragraphs between the first and last mention of "abortion"
    start_index = abortion_indices[0]
    end_index = abortion_indices[-1]
    
    paragraphs_between_abortion = all_paragraphs[start_index:end_index + 1]

    # Create a DataFrame with columns for 'name', 'year', 'dem/rep', 'left/right', 'content'
    data = {'name': [], 'year': [], 'dem/rep': [], 'left/right': [], 'content': []}

    current_speaker = None

    for paragraph in paragraphs_between_abortion:
        # Skip paragraphs that start with "MODERATOR::"
        if not paragraph.get_text().startswith("MODERATOR:"):
            # Determine 'name' and 'dem/rep' based on the speaker
            if 'BUSH:' in paragraph.get_text():
                current_speaker = 'BUSH'
            elif 'GORE:' in paragraph.get_text():
                current_speaker = 'GORE'
            
            data['name'].append(current_speaker)
            data['year'].append(2000)
            data['dem/rep'].append('rep' if current_speaker == 'BUSH' else 'dem')
            data['left/right'].append('right' if current_speaker == 'BUSH' else 'left')
            data['content'].append(re.sub(r'^BUSH:|GORE:', '', paragraph.get_text()))

    prez_df_2000 = pd.DataFrame(data)



prez_merged = pd.concat([prez_df_2020, prez_df_2016, 
                         prez_df_2016_2, prez_df_2012, 
                         prez_df_2008, prez_df_2004_2, 
                         prez_df_2004, prez_df_2000], ignore_index=True)
display(prez_merged)
print(prez_merged.shape)

,name,year,dem/rep,left/right,content
0,PENCE,2020,rep,right,"Well thank you for the question, but I’ll use..."
1,PENCE,2020,rep,right,"My hope is that when the hearing takes place,..."
2,PENCE,2020,rep,right,– treated respectfully and voted and confirme...
3,HARRIS,2020,dem,left,"Thank you, Susan. First of all, Joe Biden and..."
4,PENCE,2020,rep,right,"Well, thank you, Susan. Let me just say, addr..."
...,...,...,...,...,...
135,BUSH,2004,rep,right,"Take, for example, the ban on partial birth ab..."
136,BUSH,2004,rep,right,"What I’m saying is, is that as we promote life..."
137,BUSH,2004,rep,right,"The last debate, my opponent said his wife was..."
138,BUSH,2000,rep,right,I don’t think a president can do that. I was ...


(140, 5)


### Scraping Congressional Bills

In [14]:
def scrape_and_combine_html(urls):
    # Initialize an empty string to store the combined HTML content
    combined_html_content = ''

    # Iterate through each URL in the list
    for url in urls:
        # Fetch the HTML content from the current URL
        response = requests.get(url)
        html_content = response.content

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'xml')

        # Extract text content from the HTML
        text_content = soup.get_text(separator=' ', strip=True)

        # Append the extracted text content to the result
        combined_html_content += text_content + ' '

    return combined_html_content

#Republican
rep_congress_urls = [
    "https://www.congress.gov/118/bills/hr106/BILLS-118hr106ih.xml",
    "https://www.congress.gov/118/bills/hr7/BILLS-118hr7ih.xml",
    "https://www.congress.gov/118/bills/hr862/BILLS-118hr862ih.xml",
    "https://www.congress.gov/118/bills/hr1143/BILLS-118hr1143ih.xml",
    "https://www.congress.gov/118/bills/hr792/BILLS-118hr792ih.xml",
    "https://www.congress.gov/118/bills/hr330/BILLS-118hr330ih.xml",
    "https://www.congress.gov/118/bills/hr1470/BILLS-118hr1470ih.xml",
    "https://www.congress.gov/118/bills/hr384/BILLS-118hr384ih.xml",
    "https://www.congress.gov/118/bills/hr26/BILLS-118hr26pcs.xml",
    "https://www.congress.gov/118/bills/hr632/BILLS-118hr632ih.xml",
    "https://www.congress.gov/118/bills/hr1297/BILLS-118hr1297ih.xml",
    "https://www.congress.gov/118/bills/hr73/BILLS-118hr73ih.xml",
    "https://www.congress.gov/118/bills/hr416/BILLS-118hr416ih.xml",
    "https://www.congress.gov/118/bills/hr383/BILLS-118hr383ih.xml",
    "https://www.congress.gov/118/bills/hr5806/BILLS-118hr5806ih.xml",
    "https://www.congress.gov/118/bills/hr5319/BILLS-118hr5319ih.xml",
    "https://www.congress.gov/118/bills/hr6459/BILLS-118hr6459ih.xml",
    "https://www.congress.gov/118/bills/hr6460/BILLS-118hr6460ih.xml",
    "https://www.congress.gov/118/bills/hr3741/BILLS-118hr3741ih.xml",
    "https://www.congress.gov/118/bills/hr4672/BILLS-118hr4672ih.xml",
    "https://www.congress.gov/118/bills/hr983/BILLS-118hr983ih.xml",
    "https://www.congress.gov/118/bills/hr3741/BILLS-118hr3741ih.xml",
    "https://www.congress.gov/118/bills/hr421/BILLS-118hr421ih.xml",
    "https://www.congress.gov/118/bills/hr435/BILLS-118hr435ih.xml",
    "https://www.congress.gov/118/bills/hr175/BILLS-118hr175ih.xml",
    "https://www.congress.gov/118/bills/hr116/BILLS-118hr116ih.xml"
    # Add more HTML URLs as needed
]

rep_congress_content = scrape_and_combine_html(rep_congress_urls )


#Democratic
dem_congress_urls = [
    "https://www.congress.gov/118/bills/hr767/BILLS-118hr767ih.xml",
    "https://www.congress.gov/118/bills/hr2573/BILLS-118hr2573ih.xml",
    "https://www.congress.gov/118/bills/hr55/BILLS-118hr55ih.xml",
    "https://www.congress.gov/118/bills/hr4303/BILLS-118hr4303ih.xml",
    "https://www.congress.gov/118/bills/hr1723/BILLS-118hr1723ih.xml",
    "https://www.congress.gov/118/bills/hr561/BILLS-118hr561ih.xml",
    "https://www.congress.gov/118/bills/hr3132/BILLS-118hr3132ih.xml",
    "https://www.congress.gov/118/bills/hr12/BILLS-118hr12ih.xml",
    "https://www.congress.gov/118/bills/hr782/BILLS-118hr782ih.xml",
    "https://www.congress.gov/118/bills/hr1224/BILLS-118hr1224ih.xml",
    "https://www.congress.gov/118/bills/hr2736/BILLS-118hr2736ih.xml",
    "https://www.congress.gov/118/bills/hr6357/BILLS-118hr6357ih.xml",
    "https://www.congress.gov/118/bills/hr4796/BILLS-118hr4796ih.xml",
    "https://www.congress.gov/118/bills/hr4147/BILLS-118hr4147ih.xml",
    "https://www.congress.gov/118/bills/hr4268/BILLS-118hr4268ih.xml",
    "https://www.congress.gov/118/bills/hr4418/BILLS-118hr4418ih.xml",
    "https://www.congress.gov/118/bills/hr2907/BILLS-118hr2907ih.xml",
    "https://www.congress.gov/118/bills/hr445/BILLS-118hr445ih.xml",
    "https://www.congress.gov/118/bills/hr459/BILLS-118hr459ih.xml",
    "https://www.congress.gov/118/bills/hr4281/BILLS-118hr4281ih.xml",
    "https://www.congress.gov/118/bills/hr5008/BILLS-118hr5008ih.xml",
    "https://www.congress.gov/118/bills/hr3659/BILLS-118hr3659ih.xml",
    "https://www.congress.gov/118/bills/hr3421/BILLS-118hr3421ih.xml",
    "https://www.congress.gov/118/bills/hr6298/BILLS-118hr6298ih.xml",
    "https://www.congress.gov/118/bills/hr4901/BILLS-118hr4901ih.xml",
    "https://www.congress.gov/118/bills/hr3420/BILLS-118hr3420ih.xml", 
    "https://www.congress.gov/118/bills/hr4121/BILLS-118hr4121ih.xml",  
    "https://www.congress.gov/118/bills/hr62/BILLS-118hr62ih.xml",
    "https://www.congress.gov/118/bills/hr6298/BILLS-118hr6298ih.xml", 
    "https://www.congress.gov/118/bills/hr6270/BILLS-118hr6270ih.xml", 
    "https://www.congress.gov/118/bills/hr4329/BILLS-118hr4329ih.xml"

    # Add more HTML URLs as needed
]

dem_congress_content= scrape_and_combine_html(dem_congress_urls )

In [15]:
def get_top_words(content, custom_stopwords, top_n=10):
    # Create a CountVectorizer
    vectorizer = CountVectorizer(
        stop_words=stopwords.words('english') + custom_stopwords
    )

    # Fit and transform the content
    word_matrix = vectorizer.fit_transform([content])

    # Get feature names (words)
    words = vectorizer.get_feature_names_out()

    # Get word counts
    word_counts = word_matrix.toarray()[0]

    # Create a DataFrame with words and counts
    word_df = pd.DataFrame({'word': words, 'count': word_counts})

    # Sort DataFrame by count in descending order
    word_df = word_df.sort_values(by='count', ascending=False)

    return word_df.head(top_n)



In [16]:
# Custom stopwords for Republicans
rep_custom_stopwords = ['mr', 'abortion', 'section', 'title', 'act', 
                        'subsections', 'shall', 'following', 'code', 
                        '2023','subsection', 'mrs', 'may', 'house', 
                        'representatives', 'amended', 'bill', 'term',
                        'state', 'states', 'minor', 'means']

# Call the function for Republicans
congress_rep_top = get_top_words(rep_congress_content, rep_custom_stopwords)

congress_rep_top

,word,count
786,health,101
152,abortions,97
371,child,89
1588,united,89
1646,woman,66
1157,physical,65
691,federal,62
1199,pregnancy,58
1270,public,57
1142,performed,55


In [17]:
# Custom stop words
dem_custom_stopwords = ['mr', 'ms', 'including', 'act', 'secretary', 
                        'title', 'section', 'subsection', 'shall', '42',
                        'paragraph', 'described', 'may', 'information',
                        'state', 'states']

# Call the function for Democrats
congress_dem_top = get_top_words(dem_congress_content, dem_custom_stopwords)

congress_dem_top

,word,count
2094,health,1133
925,care,840
3685,services,665
453,abortion,514
3492,reproductive,315
3270,provider,265
2262,individual,242
4140,united,240
2265,individuals,223
1715,entity,216


In [18]:
# Define custom stopwords for Democrats
dem_stopwords = ['president', 'think', 'people', 'wade', 'said', 
                 'make', 'would', 'know', 'abortion', 'one', 'let',
                 'going', 'roe']

# Filter rows where 'dem/rep' is 'dem'
dem_df = prez_merged[prez_merged['dem/rep'] == 'dem']

# Combine all content into a single string
left_content = ' '.join(dem_df['content'])

# Call the function for Democrats
prez_dem_top = get_top_words(left_content, dem_stopwords)

prez_dem_top

,word,count
689,women,22
534,right,16
233,faith,15
358,life,15
158,court,12
322,issue,12
688,woman,12
109,catholic,12
116,choice,11
168,decision,10


In [19]:
# Define custom stopwords for Republicans
rep_stopwords = ['abortion', 'know', 'think', 'would', 'people', 
                 'senator', 'abortions', 'one', 'say', 'really',
                 'way', 'going', 'states']


# Filter rows where 'dem/rep' is 'rep'
rep_df = prez_merged[prez_merged['dem/rep'] == 'rep']

# Combine all content into a single string
right_content = ' '.join(rep_df['content'])

# Call the function for Republicans
prez_rep_top = get_top_words(right_content, rep_stopwords)

prez_rep_top

,word,count
306,life,45
31,america,16
415,pro,15
133,court,12
77,birth,10
420,promote,10
576,voted,10
32,american,9
297,law,9
99,child,9


### LABEL CREATION FROM OVERLAP

In [20]:
#RIGHT LABELS

# Extract sets of unique words from DataFrames
congress_rep_set = set(congress_rep_top['word'])
prez_rep_set = set(prez_rep_top['word'])

# Combine the sets into a single set of labels
combined_right = congress_rep_set.union(prez_rep_set)

# Print the combined set of labels
# print(combined_right)

len(combined_right)


19

In [21]:
#LEFT LABELS

# Extract sets of unique words from DataFrames
congress_dem_set = set(congress_dem_top['word'])
prez_dem_set = set(prez_dem_top['word'])

# Combine the sets into a single set of labels
combined_left = congress_dem_set.union(prez_dem_set)

# Print the combined set of labels
# print(combined_left)

len(combined_left)

20

## Creating Left and Right Labels Within All News Outlets

### Scrape in all News Outlets

In [22]:
def read_txt_to_dataframe(file_path):
    # Initialize empty lists to store data
    news_outlets = []
    titles = []
    authors = []
    publication_dates = []
    article_contents = []

    # Open the text file
    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Initialize variables to store information
        current_article = {}
        
        # Initialize list to store dictionaries
        articles_data = []
        
        # Iterate through lines in the file
        for line in lines:
            # Split the line into key and value if possible
            line_parts = line.split(':', 1)
            
            # Check if the line can be split into key and value
            if len(line_parts) == 2:
                key, value = map(str.strip, line_parts)
                
                # Check for the end of an article
                if key == 'Article_Content':
                    # Save the current article information
                    current_article['Article_Content'] = value.strip()
                    
                    articles_data.append({
                        'News_Outlet': current_article.get('News_Outlet', ''),
                        'Title': current_article.get('Title', ''),
                        'Author': current_article.get('Author', ''),
                        'Publication_Date': current_article.get('Publication_Date', ''),
                        'Article_Content': current_article.get('Article_Content', '')
                    })

                    current_article = {}
                else:
                    # Add key-value pair to current article dictionary
                    current_article[key] = value

    df = pd.DataFrame(articles_data)

    return df

In [23]:
cnn_data_frame = read_txt_to_dataframe('CNN.txt')
cnn_data_frame = cnn_data_frame.reset_index()
cnn_data_frame = cnn_data_frame.rename(columns={'index': 'docid'})
print(len(cnn_data_frame))

nyt_data_frame = read_txt_to_dataframe('NYT.txt')
nyt_data_frame = nyt_data_frame.reset_index()
nyt_data_frame = nyt_data_frame.rename(columns={'index': 'docid'})
print(len(nyt_data_frame))

fox_data_frame = read_txt_to_dataframe('FOX.txt')
fox_data_frame = fox_data_frame.reset_index()
fox_data_frame = fox_data_frame.rename(columns={'index': 'docid'})
print(len(fox_data_frame))

wsj_data_frame = read_txt_to_dataframe('WSJ.txt')
wsj_data_frame = wsj_data_frame.reset_index()
wsj_data_frame = wsj_data_frame.rename(columns={'index': 'docid'})
print(len(wsj_data_frame))

nyp_data_frame = read_txt_to_dataframe('NYP.txt')
nypdata_frame = nyp_data_frame.reset_index()
nyp_data_frame = nyp_data_frame.rename(columns={'index': 'docid'})
print(len(nyp_data_frame))


50
50
50
50
50


In [24]:
merged_outlets = pd.concat([cnn_data_frame, 
                              nyt_data_frame, 
                              fox_data_frame, 
                              wsj_data_frame, 
                              nyp_data_frame], 
                              ignore_index=True)

merged_content = merged_outlets['Article_Content']


In [25]:
# Create a new column 'binary_label' for classification (Left, Right, Neutral)
merged_outlets['party_label'] = 'Neutral'  # Default label

# Iterate over rows and label based on the distribution of words
for index, row in merged_outlets.iterrows():
    left_count = sum(word in row['Article_Content'] for word in combined_left)
    right_count = sum(word in row['Article_Content'] for word in combined_right)

    if left_count > right_count:
        merged_outlets.at[index, 'party_label'] = 'Left'
    elif right_count > left_count:
        merged_outlets.at[index, 'party_label'] = 'Right'

print(len(merged_outlets[merged_outlets['party_label'] == 'Neutral']))
print(len(merged_outlets[merged_outlets['party_label'] == 'Left']))
print(len(merged_outlets[merged_outlets['party_label'] == 'Right']))


74
102
74


### Baseline Classifiers

In [26]:
# Stratify and Split Raw Data into Train and Test groups
train_texts, test_texts, train_labels, test_labels = train_test_split(
    merged_outlets['Article_Content'], merged_outlets['party_label'], 
    test_size=0.2, random_state=None,
    stratify=merged_outlets['party_label'])

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_texts)
test_vectors = vectorizer.transform(test_texts)

# Feature selection with SelectKBest and f_classif
k_best = 20  # Adjust as needed
selector = SelectKBest(f_classif, k=k_best)
train_vectors_selected = selector.fit_transform(train_vectors, train_labels)
test_vectors_selected = selector.transform(test_vectors)

# Convert sparse matrix to dense array
train_vectors_dense = train_vectors_selected.toarray()
test_vectors_dense = test_vectors_selected.toarray()

# Train the Logistic Regression model
lg_class = LogisticRegression(max_iter=1000)
log_reg = lg_class.fit(train_vectors_dense, train_labels)
log_predictions = log_reg.predict(test_vectors_dense)

# Print classification report
print('Logistic Regression Baseline Accuracy Score: ', accuracy_score(test_labels, log_predictions))
print(classification_report(test_labels, log_predictions, zero_division=0))

# Calculate and Print Cross Validated Score for SelectKBest of Logistic Regression
lg_cv_score = cross_val_score(lg_class, train_vectors_dense, train_labels, cv=5, scoring='f1_weighted').mean()
print("Cross-validation score of Logistic Regression: ", lg_cv_score, '\n\n')

# Random Forest Baseline
rf_class = RandomForestClassifier()
ran_forest = rf_class.fit(train_vectors_dense, train_labels)
ran_forest_predict = ran_forest.predict(test_vectors_dense)
print('Random Forest Baseline Accuracy Score: ', accuracy_score(test_labels, ran_forest_predict))
print(classification_report(test_labels, ran_forest_predict))

# Calculate and Print Cross Validated Score for Random Forest
rf_cv_score = cross_val_score(rf_class, train_vectors_dense, train_labels, cv=5, scoring='f1_weighted').mean()
print("Cross validation score of Random Forest: ", rf_cv_score)


Logistic Regression Baseline Accuracy Score:  0.42
              precision    recall  f1-score   support

        Left       0.41      0.85      0.56        20
     Neutral       0.44      0.27      0.33        15
       Right       0.00      0.00      0.00        15

    accuracy                           0.42        50
   macro avg       0.29      0.37      0.30        50
weighted avg       0.30      0.42      0.32        50

Cross-validation score of Logistic Regression:  0.4155761492309867 




Random Forest Baseline Accuracy Score:  0.54
              precision    recall  f1-score   support

        Left       0.54      0.75      0.63        20
     Neutral       0.75      0.20      0.32        15
       Right       0.50      0.60      0.55        15

    accuracy                           0.54        50
   macro avg       0.60      0.52      0.50        50
weighted avg       0.59      0.54      0.51        50

Cross validation score of Random Forest:  0.5355208992895877


### Bert Classifier

In [32]:
#Design, implement, and evaluate the performance of a classifier based on 
#(Distil)BERT or other large language model. Measure performance using one or 
#more metrics directly comparable to the one(s) used for your baseline method.

max_length = 512  # Set the maximum length for tokenization

# Tokenize the poems using DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

# Define the number of folds for cross-validation
n_splits = 3  

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=n_splits, random_state=None, shuffle=True)

# Custom dataset class for PyTorch, handling encodings and labels
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# # Training arguments for the Hugging Face Transformers library
# training_args = TrainingArguments(
#     output_dir='./results',         # Outputs directory
#     logging_dir='./logs',           # Directory for storing logs
#     per_device_train_batch_size=9,  # Set the batch size per device for training
#     per_device_eval_batch_size=8,   # Set the batch size for evaluation
#     num_train_epochs=4,             # Total Number of training epochs
#     learning_rate=1e-4,             # Initial learning rate for Adam optimizer
#     warmup_steps=4,                 # Number of warm-up steps for learning rate scheduler
#     weight_decay=0.005,             # Weight decay strength
#     logging_steps=6,               # Logging steps within directory during training
#     evaluation_strategy='steps'     # Evaluate steps during fine-tuning to see progress
# )
    
training_args = TrainingArguments(
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=200,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.1,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

# Function to compute evaluation metrics for model predictions
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    score1 = accuracy_score(labels, preds)
    score = f1_score(labels, preds, average='weighted')
    return {'f1': score, 'accuracy': score1}




In [31]:
# Initialize lists to store information from each fold and prediction
fold_results = []   # List to store information from each fold
f1_score_list = []  # List to store F1 scores for each fold
accuracy_list = []  # List to store accuracy scores for each fold

# Initialize misclassifications dictionary
party_classifications_dict = defaultdict(int)

# Iterate over folds using StratifiedKFold
for i, (train_index, test_index) in enumerate(skf.split(merged_outlets['Article_Content'], merged_outlets['party_label'])):
    print(f"Fold {i}:")
    
    # Split the data into train and test sets for the current fold
    train_data_fold = merged_outlets.iloc[train_index]
    test_data_fold = merged_outlets.iloc[test_index]

    train_texts_fold = train_data_fold['Article_Content'].tolist()
    train_labels_fold = train_data_fold['party_label'].tolist()

    test_texts_fold = test_data_fold['Article_Content'].tolist()
    test_labels_fold = test_data_fold['party_label'].tolist()

    print('Lengths of train and tests texts and labels: ') 
    print('Train texts: ', len(train_texts_fold))
    print('Train labels: ', len(train_labels_fold))
    print('Test texts: ', len(test_texts_fold))
    print('Test labels: ', len(test_labels_fold))

    # Define label2id and id2label based on unique labels in the current fold
    unique_labels = set(train_labels_fold + test_labels_fold)
    label2id = {label: id for id, label in enumerate(unique_labels)}
    id2label = {id: label for label, id in label2id.items()}

    # Tokenize the articles using DistilBertTokenizerFast for the current fold
    train_encodings_fold = tokenizer(list(train_texts_fold), truncation=True, padding=True, max_length=512)
    test_encodings_fold = tokenizer(list(test_texts_fold), truncation=True, padding=True, max_length=512)

    train_labels_encoded_fold = [label2id[y] for y in train_labels_fold]
    test_labels_encoded_fold = [label2id[y] for y in test_labels_fold]

    # Create datasets for the current fold
    train_dataset_fold = MyDataset(train_encodings_fold, train_labels_encoded_fold)
    test_dataset_fold = MyDataset(test_encodings_fold, test_labels_encoded_fold)

    # Create and train the model for the current fold using DistilBertForSequenceClassification
    model_fold = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=len(id2label)).to('cpu')

    trainer_fold = Trainer(
        model=model_fold,                    # instantiated Transformers model to be trained
        args=training_args,                  # Training arguments initialized above
        train_dataset=train_dataset_fold,    # Training dataset
        eval_dataset=test_dataset_fold,      # Evaluation dataset
        compute_metrics=compute_metrics      # Custom evaluation function
    )

    trainer_fold.train()
    
    trainer_fold.evaluate()

    # Get predictions on the test set
    predictions_fold = trainer_fold.predict(test_dataset_fold)

    predicted_labels = predictions_fold.predictions.argmax(-1)  # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
    predicted_labels_num = [label2id[label] for label in predicted_labels]

    print('Length of Predicted Labels: ', len(predicted_labels))
    print('Printed Predicted Labels: ', predicted_labels[:21])
    print('\nClassification Report: ')
    print(classification_report(test_labels_encoded_fold, predicted_labels_num))

    # Calculate and store F1 score and accuracy for the current fold
    f1_score_list.append(f1_score(test_labels_encoded_fold, predicted_labels_num, average='weighted'))
    accuracy_list.append(accuracy_score(test_labels_encoded_fold, predicted_labels_num))

    # Extract information for the current fold and store it in the overall list
    fold_info = [(id2label[true_label], id2label[predicted_label], text) for true_label, predicted_label, text in zip(test_labels_encoded_fold, predicted_labels_num, test_texts_fold)]
    fold_results.append(fold_info)
    
    # If it's the last fold, update tag_classifications_dict
    if i == (n_splits - 1):
        for true_label, predicted_label, _ in fold_info:
            if true_label != predicted_label:
                party_classifications_dict[(true_label, predicted_label)] += 1


Fold 0:
Lengths of train and tests texts and labels: 
Train texts:  200
Train labels:  200
Test texts:  50
Test labels:  50


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/100 [00:00<?, ?it/s]

{'loss': 1.0815, 'learning_rate': 1e-05, 'epoch': 5.0}


  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

Length of Predicted Labels:  50
Printed Predicted Labels:  ['Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left']

Classification Report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.00      0.00      0.00        15
           2       0.40      0.95      0.56        20

    accuracy                           0.38        50
   macro avg       0.13      0.32      0.19        50
weighted avg       0.16      0.38      0.22        50

Fold 2:
Lengths of train and tests texts and labels: 
Train texts:  200
Train labels:  200
Test texts:  50
Test labels:  50


/Applications/anaconda3/envs/3350 /lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/envs/3350 /lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/envs/3350 /lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 